In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as i

In [2]:
#Assigning Parameters
G=6.674*10**(-11)  #Gravitational Constant
M=5.972*10**24     #Mass of the Earth
sea=6378000        #distance from the center of the Earth to sea level
karman=sea+100000  #distance from the center of the Earth to the Karman line
R=287.05           #Ideal Gas Constant
Area=40.47         #Vross-sectional area of the first stage rocket
Cd=0.7             #Drag Coefficient for the first stage rocket
maxThrust=24681000 #Maximum thrust of the first stage of the rocket in space

payload=63800                 #The mass of the payload
f_mi=1420788-(63800-payload)    #The initial mass of the rocket
f_ifuelm=414000*3               #The initial mass of the fuel
f_fuelm = f_ifuelm
f_drym=f_mi-f_ifuelm                #The dry mass of the rocket

#Second Stage
s_ifuelm=97000               #Redefining the initial fuel mass for the second stage
s_drym=4000                  #This is the same as dry mass for the second stage
s_fuelm=s_ifuelm            #The fuelmass of the rocket is assigned to its initial value


vx=0                          #The initial x-velocity
vy=0.01                       #The initial y-velocity isn't set to zero to avoid dividing by zero
ry=sea                        #Initial y distance from the center of the Earth
rx=0 

In [3]:
def Rocket(z,t,dt,xT,yT):
    rx=z[0]
    ry=z[1]
    vx=z[2]
    vy=z[3]
    ax=0                          #Initial x-acceleration
    ay=0                          #Initial y-acceleration
    r=np.sqrt(rx**2+ry**2)        #Setting the magnitude of the distance
    Drag=0                        #Initial drag force
    
    r=np.sqrt(rx**2+ry**2)     #Calculating the magnitude of distance from its components
    v=np.sqrt(vx**2+vy**2)     #Calculating the magnitude of velocity from its components
    a=np.sqrt(ax**2+ay**2)     #Calculating the magnitude of acceleration from its components
    
    if r<sea:
        r=sea
        v=0
        a=0
        vx=0
        vy=0
        ax=0
        ay=0
        
    #The Altitude changes pressure, density, and temperature
    if (r-sea)<11000:
        Temp = (15.04-0.00649*(r-sea))+ 273.1
        p=101290*(Temp/288.08)**5.256
        den=p/(R*Temp)
    if 11000<=(r-sea)<=25000:
        Temp = -56.46 + 273.1 
        p =22650*np.exp(1.73-0.000157*(r-sea))
        den=p/(R*Temp)
    if (r-sea)>25000:
        Temp = (-131.21 + .00299 * (r-sea) )+ 273.1
        p = 2488 * (Temp/ 216.6)**(-11.388)
        den=p/(R*Temp)
    if (r-sea)>100000: #if above the Karman Line
        den=0
        p=0
    
    Thrust=maxThrust-18.3765*p                      #The total thrust changes as the air pressure changes
    XThrust=xT * Thrust
    YThrust=yT * Thrust

        
    throttle = np.sqrt(xT**2 + yT**2)
    f_fuelm=f_fuelm-(throttle*dt*(f_ifuelm/162))
    m=f_drym+f_fuelm
    
    Fg=(G*m*M)/(r**2)    #The force of gravity
    Fgx=Fg*(rx/r)        #The x-component of gravity based off of the position vector
    Fgy=Fg*(ry/r)        #The y-component of gravity based off of the position vector
    

    Dx=-Cd*den*Area*v*vx/2       #The x-drag (air friction) is based off of its x-velocity, and air density
    Dy=-Cd*den*Area*v*vy/2       #The y-drag (air friction) is based off of its y-velocity, and air density
    Drag=np.sqrt(Dx**2+Dy**2)    #Calculating the total drag of the rocket
    
    ax=(XThrust-Fgx+Dx)/m       #The x-acceleration is adding the x-forces and dividing by the rockets mass
    ay=(YThrust-Fgy+Dy)/m       #The y-acceleration is adding the y-forces and dividing by the rockets mass  
    
    if f_fuelm <= 0:
        maxThrust=934000              #The maximum thrust of the rocket in space for the second stage engines
        m=payload+s_fuelm+s_drym  #Recalculating the mass of the rocket
        Area=21.237                   #The new cross-sectional area of the rocket since the booster rockets were detached
        Cd=.5                         #The new drag coefficient of the rocket

        r=np.sqrt(rx**2+ry**2)     #Calculating the new position
        v=np.sqrt(vx**2+vy**2)     #Calculating the new velocity
        a=np.sqrt(ax**2+ay**2)     #Calculating the new acceleration
        
        Thrust=maxThrust-18.3765*p    #The simplified Rocket Thrust Equation changes the thrust due to air pressure
        XThrust=xT * Thrust           #All the thrust needs to be in the x-direction so the rocket can accelerate
        YThrust=yT * Thrust             #itself into orbit and build up enough speed
        
        #Recalculating the mass of the rocket
        s_fuelm=s_fuelm-(throttle*dt*(s_ifuelm/397))  #Calculating the new fuelmass since fuel was burnt
        m=payload+s_fuelm+s_drym                                  #New mass of the rocket
        
        Dx=-Cd*den*Area*v*vx/2       #Calculating air friction for x-component
        Dy=-Cd*den*Area*v*vy/2        #Calculating air friction for y-component
        Drag=np.sqrt(Dx**2+Dy**2)    #Calculating magnitude of air friction
        
        if s_fuelm <= 0:        #If the rocket doesn't have any more fuel
            m=payload+inertmass  #The mass is simply the payload and the drymass
            YThrust=0            #Don't thrust anywhere
            XThrust=0            #There is no more fuel to burn
            
        Fg=(G*m*M)/(r**2)      #Calculating the magnetude of gravity
        Fgx=Fg*(rx/r)          #Calculating the x-component of gravity
        Fgy=Fg*(ry/r)          #Calculating the y-component of gravity
        
        ax=(XThrust-Fgx+Dx)/m  #Updating the x-acceleration by adding the forces and dividing by the mass
        ay=(YThrust-Fgy+Dy)/m  #Updating the y-acceleration by adding the forces and dividing by the mass
        
    
    
    k1 = h * f(t, x)
    k2 = h * f(t+h/2, x+k1/2)
    k3 = h * f(t+h/2, x+k2/2)
    k4 = h * f(t+h, x+k3)
    
    x = x + 1/6 * (k1 + 2*k2 + 2*k3 + k4)
    
    
    zz = [vx,vy,ax,ay]

    return zz

In [4]:
z = [rx, ry, vx, vy]
dt=0.01
time = np.arange(0,10000,dt)
xT=0
yT=1


UnboundLocalError: local variable 'maxThrust' referenced before assignment